<a href="https://colab.research.google.com/github/Peggypei98/NLP-for-Health/blob/main/Peggy_new_BOW_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Check GPU
# !nvidia-smi

Mon Apr  1 09:10:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')


%cd '/content/drive/Shareddrives/NLP_for_Health'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/NLP_for_Health


In [ ]:
import re
import nltk

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf

In [ ]:
# Data prep
import pandas as pd
test_data = pd.read_csv("/content/drive/Shareddrives/NLP_for_Health/Code/Data/Data2/dreddit-test.csv", encoding = 'utf-8')
train_data = pd.read_csv("/content/drive/Shareddrives/NLP_for_Health/Code/Data/Data2/dreddit-train.csv", encoding = 'utf-8')

combined_df_vertical = pd.concat([test_data, train_data], ignore_index=True)
combined_df_vertical.to_csv('/content/drive/Shareddrives/NLP_for_Health/Code/Data/Data2/new_dereddit.csv', index=False)

df = pd.read_csv("/content/drive/Shareddrives/NLP_for_Health/Code/Data/Data2/new_dereddit.csv", encoding = 'utf-8')
df.head()

,id,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,896,relationships,7nu7as,"[50, 55]","Its like that, if you want or not.“ ME: I have...",0,0.8,1.514981e+09,22,-1.238793,...,1.0000,1.2000,1.0,1.65864,1.32245,1.80264,0.63,62,-0.148707,0.000000
1,19059,anxiety,680i6d,"(5, 10)",I man the front desk and my title is HR Custom...,0,1.0,1.493348e+09,5,7.684583,...,1.4000,1.1250,1.0,1.69133,1.69180,1.97249,1.00,2,7.398222,-0.065909
2,7977,ptsd,8eeu1t,"(5, 10)",We'd be saving so much money with this new hou...,1,1.0,1.524517e+09,10,2.360408,...,1.1429,1.0000,1.0,1.70974,1.52985,1.86108,1.00,8,3.149288,-0.036818
3,1214,ptsd,8d28vu,"[2, 7]","My ex used to shoot back with ""Do you want me ...",1,0.5,1.524018e+09,5,5.997000,...,1.0000,1.3000,1.0,1.72615,1.52000,1.84909,1.00,7,6.606000,-0.066667
4,1965,relationships,7r1e85,"[23, 28]",I haven’t said anything to him yet because I’m...,0,0.8,1.516200e+09,138,4.649418,...,1.1250,1.1429,1.0,1.75642,1.43582,1.91725,0.84,70,4.801869,0.141667


In [ ]:
df.describe(include='object')

,subreddit,post_id,sentence_range,text
count,3553,3553,3553,3553
unique,10,2929,194,3532
top,ptsd,9md6qz,"[0, 5]","Hello, You are invited to complete a survey fo..."
freq,711,6,431,4


In [ ]:
df.dtypes

id                            int64
subreddit                    object
post_id                      object
sentence_range               object
text                         object
                             ...   
lex_dal_avg_pleasantness    float64
social_upvote_ratio         float64
social_num_comments           int64
syntax_fk_grade             float64
sentiment                   float64
Length: 116, dtype: object

In [ ]:
# Count missing values for each column
missing_per_column = df.isnull().sum()
print("Missing values per column:")
print(missing_per_column)

# Total count of missing values in the DataFrame
total_missing = df.isnull().sum().sum()
print("\nTotal missing values in DataFrame:", total_missing)

Missing values per column:
id                          0
subreddit                   0
post_id                     0
sentence_range              0
text                        0
                           ..
lex_dal_avg_pleasantness    0
social_upvote_ratio         0
social_num_comments         0
syntax_fk_grade             0
sentiment                   0
Length: 116, dtype: int64

Total missing values in DataFrame: 0


In [ ]:
# ! pip install -U nltk==3.5

In [ ]:
# Remove stop words, punctuations, links, direct calls to subreddits
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')


def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Delete link
    text = re.sub(r'/r/\w+', '', text)   # Remove direct calls to subforums
    text = re.sub(r'[^\w\s]', '', text)  # Delete punctuation
    text = re.sub(r'r/[^\s]+', '', text) # Remove subreddit mentions
    stop_words = set(stopwords.words('english')) # Delete stop words
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if not word in stop_words]

    # Rejoin words
    text = ' '.join(filtered_text)
    return text

df['processed_text'] = df['text'].apply(preprocess_text)

print(df[['text', 'processed_text']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                text  \
0  Its like that, if you want or not.“ ME: I have...   
1  I man the front desk and my title is HR Custom...   
2  We'd be saving so much money with this new hou...   
3  My ex used to shoot back with "Do you want me ...   
4  I haven’t said anything to him yet because I’m...   

                                      processed_text  
0  Its like want ME I problem takes longer But as...  
1  I man front desk title HR Customer Service Rep...  
2  Wed saving much money new housrits expensive c...  
3  My ex used shoot back Do want go time matter I...  
4  I havent said anything yet Im sure someone wou...  


In [ ]:
# Keyword extration using TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np

texts = df["processed_text"]

vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)
tfidf_norm = normalize(tfidf_matrix, norm='l1', axis=1)
feature_names = np.array(vectorizer.get_feature_names_out())

# Function to get the top n keywords for each document
def get_top_n_keywords(tfidf, features, top_n=3):
    sorted_indices = np.argsort(tfidf.toarray(), axis=1)[:,::-1]
    top_n_indices = sorted_indices[:,:top_n]
    top_n_keywords = features[top_n_indices]

    return top_n_keywords

# Get the top 10 keywords for each document
top_keywords = get_top_n_keywords(tfidf_norm, feature_names, top_n=10)

# Print the top 10 keywords for each document
for i, keywords in enumerate(top_keywords):
    print(f"Document {i+1}: {', '.join(keywords)}")

df['keywords'] = ['; '.join(keywords) for keywords in top_keywords]


Document 1: hours, 5am, asked, prepared, till, wait, takes, hour, instead, longer
Document 2: benefits, retirement, pension, clericalpaperwork, onboarding, hires, basics, representative, base, employees
Document 3: googling, contract, wed, mw, housrits, 6months, illegal, ill, swear, long
Document 4: champ, unloaded, chasing, shoot, phrase, circles, blocked, wedding, throw, wonder
Document 5: feelings, fluctuations, nonetheless, clingy, recommend, yall, warm, hot, hearing, somewhat
Document 6: report, verify, copy, receipt, fuel, requested, filed, incident, cold, responses
Document 7: homed, tiny, rules, state, privileges, id, lest, house, homes, volunteer
Document 8: roommates, example, overload, tangents, nonstop, storming, concentrate, gosh, hyper, lesson
Document 9: ohio, family, stay, ne, beloved, south, packing, sending, rid, cat
Document 10: sweet, super, concocted, leandro, arsenal, escort, mapped, watt, handsome, envious
Document 11: cloud, carried, nail, files, recall, underst

In [ ]:
# Word Embedding using bag of words
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df['keywords'])
df['bow_embeddings'] = list(X_bow.toarray())



In [ ]:
# Check all data and drop other columns
df = df[['subreddit', 'label', 'text', 'processed_text', 'keywords', 'bow_embeddings']]
df.head()

,subreddit,label,text,processed_text,keywords,bow_embeddings
0,relationships,0,"Its like that, if you want or not.“ ME: I have...",Its like want ME I problem takes longer But as...,hours; 5am; asked; prepared; till; wait; takes...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,anxiety,0,I man the front desk and my title is HR Custom...,I man front desk title HR Customer Service Rep...,benefits; retirement; pension; clericalpaperwo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,ptsd,1,We'd be saving so much money with this new hou...,Wed saving much money new housrits expensive c...,googling; contract; wed; mw; housrits; 6months...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,ptsd,1,"My ex used to shoot back with ""Do you want me ...",My ex used shoot back Do want go time matter I...,champ; unloaded; chasing; shoot; phrase; circl...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,relationships,0,I haven’t said anything to him yet because I’m...,I havent said anything yet Im sure someone wou...,feelings; fluctuations; nonetheless; clingy; r...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# split the data into train, test, dev
import numpy as np
from sklearn.model_selection import train_test_split

X = np.stack(df['bow_embeddings'].values)
y = df['label'].values


# Splitting the data into 85% for the initial split and 15% for the test set
train_dev, df_test = train_test_split(df, test_size=0.15, random_state=42)

# Splitting the remaining 85% into 82.35% train and 17.65% dev to achieve roughly 70-15-15 split across train-dev-test
df_train, df_dev = train_test_split(train_dev, test_size=0.1765, random_state=42)  # This ratio splits 85% into roughly 70% and 15%

print(f"Train set size: {len(df_train)}")
print(f"Test set size: {len(df_test)}")
print(f"Dev set size: {len(df_dev)}")


Train set size: 2486
Test set size: 533
Dev set size: 534


In [ ]:
X_train = np.squeeze(np.stack(df_train['bow_embeddings'].values))
y_train = df_train['label'].values

X_dev = np.squeeze(np.stack(df_dev['bow_embeddings'].values))
y_dev = df_dev['label'].values

X_test = np.squeeze(np.stack(df_test['bow_embeddings'].values))
y_test = df_test['label'].values

print(f"Shapes after squeezing: X_train: {X_train.shape}, X_dev: {X_dev.shape}, X_test: {X_test.shape}")

Shapes after squeezing: X_train: (2486, 12919), X_dev: (534, 12919), X_test: (533, 12919)


In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
# SVM
from sklearn.svm import SVC

svm_model = SVC(probability=True)  # Enable probability estimates for SVM
svm_model.fit(X_train, y_train)


SVC(probability=True)

In [ ]:
# XGBoost
import xgboost as xgb

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [ ]:
from sklearn.metrics import f1_score, recall_score, average_precision_score, accuracy_score, precision_recall_curve
from sklearn.metrics import precision_score

def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X)

    print(f"Accuracy: {accuracy_score(y, y_pred):.4f}")
    print(f"Recall (Macro-average): {recall_score(y, y_pred, average='macro'):.4f}")
    print(f"Recall (Micro-average): {recall_score(y, y_pred, average='micro'):.4f}")
    print(f"Precision (Macro-average): {precision_score(y, y_pred, average='macro'):.4f}")
    print(f"Precision (Micro-average): {precision_score(y, y_pred, average='micro'):.4f}")
    print(f"F1 Score (Macro-average): {f1_score(y, y_pred, average='macro'):.4f}")
    print(f"F1 Score (Micro-average): {f1_score(y, y_pred, average='micro'):.4f}")
    average_precision = average_precision_score(y, y_proba)
    print(f"Average Precision: {average_precision:.4f}")

# Evaluate each model
print("Evaluating Logistic Regression:")
evaluate_model(lr_model, X_dev, y_dev)

print("\nEvaluating SVM:")
evaluate_model(svm_model, X_dev, y_dev)

print("\nEvaluating XGBoost:")
evaluate_model(xgb_model, X_dev, y_dev)


Evaluating Logistic Regression:
Accuracy: 0.6180
Recall (Macro-average): 0.6177
Recall (Micro-average): 0.6180
Precision (Macro-average): 0.6177
Precision (Micro-average): 0.6180
F1 Score (Macro-average): 0.6177
F1 Score (Micro-average): 0.6180
Average Precision: 0.6820

Evaluating SVM:
Accuracy: 0.6124
Recall (Macro-average): 0.6106
Recall (Micro-average): 0.6124
Precision (Macro-average): 0.6122
Precision (Micro-average): 0.6124
F1 Score (Macro-average): 0.6101
F1 Score (Micro-average): 0.6124
Average Precision: 0.6766

Evaluating XGBoost:
Accuracy: 0.6180
Recall (Macro-average): 0.6208
Recall (Micro-average): 0.6180
Precision (Macro-average): 0.6247
Precision (Micro-average): 0.6180
F1 Score (Macro-average): 0.6158
F1 Score (Micro-average): 0.6180
Average Precision: 0.6696


In [ ]:
# Logistic regression tuning

from sklearn.model_selection import GridSearchCV

lr_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

lr_grid_search = GridSearchCV(LogisticRegression(max_iter=1000), lr_param_grid, cv=5, scoring='f1_macro')
lr_grid_search.fit(X_train, y_train)

print(f"Best parameters for Logistic Regression: {lr_grid_search.best_params_}")
print(f"Best score for Logistic Regression: {lr_grid_search.best_score_:.4f}")


Best parameters for Logistic Regression: {'C': 0.1, 'solver': 'lbfgs'}
Best score for Logistic Regression: 0.6556


In [ ]:
# SVM Tuning

svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

svm_grid_search = GridSearchCV(SVC(probability=True), svm_param_grid, cv=5, scoring='f1_macro')
svm_grid_search.fit(X_train, y_train)

print(f"Best parameters for SVM: {svm_grid_search.best_params_}")
print(f"Best score for SVM: {svm_grid_search.best_score_:.4f}")


Best parameters for SVM: {'C': 10, 'kernel': 'rbf'}
Best score for SVM: 0.6552


In [ ]:
# XGBoost Tuning
xgb_param_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1]
}

xgb_grid_search = GridSearchCV(xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
                               xgb_param_grid, cv=5, scoring='f1_macro')
xgb_grid_search.fit(X_train, y_train)

print(f"Best parameters for XGBoost: {xgb_grid_search.best_params_}")
print(f"Best score for XGBoost: {xgb_grid_search.best_score_:.4f}")


Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}
Best score for XGBoost: 0.6001


In [ ]:
# Re-evaluate models with their best parameters on the test set
print("Final Evaluation on Test Set")

print("\nEvaluating Tuned Logistic Regression:")
best_lr_model = lr_grid_search.best_estimator_
evaluate_model(best_lr_model, X_test, y_test)

print("\nEvaluating Tuned SVM:")
best_svm_model = svm_grid_search.best_estimator_
evaluate_model(best_svm_model, X_test, y_test)

print("\nEvaluating Tuned XGBoost:")
best_xgb_model = xgb_grid_search.best_estimator_
evaluate_model(best_xgb_model, X_test, y_test)


Final Evaluation on Test Set

Evaluating Tuned Logistic Regression:
Accuracy: 0.6417
Recall (Macro-average): 0.6355
Recall (Micro-average): 0.6417
Precision (Macro-average): 0.6388
Precision (Micro-average): 0.6417
F1 Score (Macro-average): 0.6357
F1 Score (Micro-average): 0.6417
Average Precision: 0.7327

Evaluating Tuned SVM:
Accuracy: 0.6623
Recall (Macro-average): 0.6598
Recall (Micro-average): 0.6623
Precision (Macro-average): 0.6600
Precision (Micro-average): 0.6623
F1 Score (Macro-average): 0.6599
F1 Score (Micro-average): 0.6623
Average Precision: 0.7321

Evaluating Tuned XGBoost:
Accuracy: 0.5966
Recall (Macro-average): 0.6072
Recall (Micro-average): 0.5966
Precision (Macro-average): 0.6134
Precision (Micro-average): 0.5966
F1 Score (Macro-average): 0.5939
F1 Score (Micro-average): 0.5966
Average Precision: 0.6737
